# Assignment 1


In this assignment, we will explore agents that can (1) find paths through a maze and (2) play TicTacToe.

There are a mix of <font color="blue">Implementation</font> cells, where you are to fill in the code for a function, as well as <font color="blue">Question</font> cells, where you are to complete short answer questions.

<br><br>

## <font color="blue">Path finding</font>

For the first part of this assignment, you'll implement several search algorithms for an agent to search a maze.

In the cell below, we have given the implementaiton of a `Board` class, which provides a few capabilities:

- It tracks which spaces have been explored.
- `shortest_path` is a dict that keeps track of the number of steps in the shorts path to reach a cell, based on the spaces that have been explored so far. 
- The `parent_of` dict keeps track of the `parent` of each space, which is the location of the preceding cell in the shortest path found so far.
- Using python [decorators](https://wiki.python.org/moin/PythonDecorators), we count how often each method is called, for getting empirical estimates of efficiency.
- Simple ascii animation of the search progress.

In [1]:
from IPython.display import clear_output
from collections import Counter, defaultdict
import math
import time
    
# Constants to represent what can be in each space.
EMPTY = '  .'     # space that has not yet been explored and is not a wall, goal, or start space
WALL = '  O'      # space that the robot can not pass through
EXPLORED = '  *'  # indicates that the robot has visited this space
GOAL = '  $'      # the goal
START = '  >'     # where the robot starts

def count_calls(fn):
    """
    Decorator function for counting how often each 
    function is called in the Board class.
    More info at https://stackoverflow.com/a/11731208
    No need to worry about how this works at the moment.
    """
    def _counting(*args, **kwargs):
        args[0].call_counts[fn.__name__] += 1
        return fn(*args, **kwargs)
    return _counting

class Board:
    def __init__(self, rows, cols, start_location,
                 goal_location, wall_locations, animate=True):
        self.rows, self.cols = rows, cols
        self.goal_location, self.start_location = goal_location, start_location
        self.cells = [[EMPTY] * cols for j in range(rows)]
        for x,y in wall_locations:
            self.cells[x][y] = WALL
        self.cells[goal_location[0]][goal_location[1]] = GOAL
        self.cells[start_location[0]][start_location[1]] = START
        self.call_counts = Counter()
        self.animate = animate
        self.parent_of = defaultdict(lambda: math.inf)
        self.shortest_path = defaultdict(lambda: math.inf)
        self.shortest_path[None] = -1
        
    def neighbors(self, location):
        """
        Return list of legal neighbor locations.
        Assumes we can move NESW.
        """
        result = []
        for (x_move, y_move) in [(-1,0), (0,1), (1,0), (0,-1)]:
            new_location = (location[0] + x_move, location[1] + y_move)
            if self.legal_location(new_location):
                result.append(new_location)
        return result

    @count_calls
    def found_goal(self):
        """
        Return True if the goal_location has been explored
        """        
        return self.cells[self.goal_location[0]][self.goal_location[1]] == EXPLORED
    
    @count_calls
    def legal_location(self, location):
        """
        Is this location in bounds and not a wall?
        """
        return (location[0] >= 0 and location[0] < self.rows and
                location[1] >= 0 and location[1] < self.cols and
                self.cells[location[0]][location[1]] != WALL)
    
    @count_calls
    def explore(self, location, parent):
        """
        Explore this location and update its shortest path.
        Optionally animate the board.
        """
        self.cells[location[0]][location[1]] = EXPLORED
        new_cost = self.shortest_path[parent] + 1
        if new_cost < self.shortest_path[location]:
            self.shortest_path[location] = new_cost
            self.parent_of[location] = parent
        if self.animate:
            clear_output(wait=True)
            self.draw()
            time.sleep(.1)

    @count_calls
    def is_explored(self, location):
        """
        return True if this location has been explored
        """
        return self.cells[location[0]][location[1]] == EXPLORED
        
    @count_calls
    def draw(self):
        """
        draw the board
        """
        for i in range(self.rows):
            print(''.join(self.cells[i]))
            
    def call_stats(self):
        """
        Print the number of times each method has been called.
        """
        print('\n'.join('%20s\t%d' % (k, v) for k, v in self.call_counts.most_common()))
        count = 0
        for r in self.cells:
            for c in r:
                if c == EXPLORED:
                    count += 1                    
        print('explored %d cells' % count)
        
    def draw_best_path(self):
        """
        Draw the shortest path found from start to goal location.
        Assumes goal has been found.
        """
        node = self.goal_location
        cost = self.shortest_path[node]
        while node is not None:
            self.cells[node[0]][node[1]] = '%3d' % cost
            cost -= 1
            node = self.parent_of[node]
        self.draw()

def example_board():    
    return Board(rows=7,
                 cols=10,
                 start_location=(3,0), 
                 goal_location=(6,7),
                 wall_locations=[(1,3), (1,4), (1,5), (1,6), (2,6), (3,6), (4,6), (5,3), (5,4), (5,5), (5,6)])
board = example_board()
board.draw()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  >  .  .  .  .  .  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  .  $  .  .


Here is a very simple, very bad example of a search algorithm that uses the `Board` class. It randomly chooses a neighboring location to explore until it finds the goal. This may take a bit to run, depending on how lucky its random guesses are.

Note that we set the parent of the start node to `None`.

In [2]:
import random

def random_search(board):
    random.seed(1234)
    location = board.start_location
    parent = None
    while not board.found_goal():
        board.explore(location, parent)
        parent = location
        location = random.choice(board.neighbors(location))

board = example_board()
# you can turn off animation to just skip to the end of the search.
# board.animate = False
random_search(board)
board.call_stats()

  *  *  *  *  *  *  *  *  *  *
  .  *  *  O  O  O  O  *  *  *
  .  *  .  .  .  .  O  *  *  *
  *  *  .  .  .  .  O  *  *  *
  .  .  .  .  .  .  O  .  *  *
  .  .  .  O  O  O  O  *  *  .
  .  .  .  .  .  .  .  *  .  .
      legal_location	484
          found_goal	122
             explore	121
                draw	121
explored 29 cells


Two things we are interested in are:
- the total number of spaces explored (indicated by the "*" symbols)
- the length of the shortest path found from start to goal

In [3]:
board.draw_best_path()

  *  4  5  6  7  8  9 10  *  *
  .  3  *  O  O  O  O 11  *  *
  .  2  .  .  .  .  O 12 13  *
  0  1  .  .  .  .  O  * 14  *
  .  .  .  .  .  .  O  . 15  *
  .  .  .  O  O  O  O 17 16  .
  .  .  .  .  .  .  . 18  .  .


<br><br>

<hr>

### <font color="blue">Question</font>

Sometimes it seems as if the search is paused (no stars are appearing). What is happening here?

<hr>

<br><br>

**answer here**
Because it randomly chooses a neighboring location. Then,there is probability 1/8 that the point will go back to the parent and then go back to this point. Just like it jump between two points in cycle. In the graph it sounds like the search is paused (no stars are appearing). 
<br><br><br><br>


<br><br>

<hr>

### <font color="blue">Implementation</font>

Implement depth first search in the `dfs` method below. You may choose to do so recursively or iteratively. Be sure to call the `board.explore` function when nodes are encountered in order to display progress and update the shortest paths.

Be sure to do *graph search* so we don't have any infinite loops.

Here is an example of what my implementation outputs, though yours may differ:

```
  *  *  *  *  *  *  *  *  *  *
  *  .  .  O  O  O  O  *  *  *
  *  .  .  .  .  .  O  *  *  *
  *  .  .  .  .  .  O  *  *  *
  .  .  .  .  .  .  O  *  *  *
  .  .  .  O  O  O  O  *  *  *
  .  .  .  .  .  .  .  *  *  *
      legal_location	124
         is_explored	51
          found_goal	50
             explore	31
                draw	31
explored 31 cells                
  3  4  5  6  7  8  9 10 11 12
  2  .  .  O  O  O  O 25 24 13
  1  .  .  .  .  .  O 26 23 14
  0  .  .  .  .  .  O 27 22 15
  .  .  .  .  .  .  O 28 21 16
  .  .  .  O  O  O  O 29 20 17
  .  .  .  .  .  .  . 30 19 18
```

In [4]:
def depth_first_search(board):
    location = board.start_location
    parent = None
    visited, stack = set(), [(location, parent)]
    while not board.found_goal() and stack:
        location, parent = stack.pop()
        if location not in visited:
            board.explore(location, parent)
            parent = location
            
            visited.add(location)
            stack.extend([(loc,parent) for loc in (set(board.neighbors(location))-visited)])
            
board = example_board()
depth_first_search(board)
board.call_stats()
board.draw_best_path()

  .  .  *  *  *  *  *  *  *  *
  .  .  *  O  O  O  O  *  *  *
  *  *  *  .  .  .  O  *  *  *
  *  .  .  .  .  .  O  *  *  *
  .  .  .  .  .  .  O  *  *  .
  .  .  .  O  O  O  O  *  *  .
  .  .  .  .  .  .  .  *  .  .
      legal_location	108
          found_goal	28
             explore	27
                draw	27
explored 27 cells
  .  .  5  6  7  8  9 10  *  *
  .  .  4  O  O  O  O 11 14 15
  1  2  3  .  .  .  O 12 13 16
  0  .  .  .  .  .  O  * 18 17
  .  .  .  .  .  .  O  * 19  .
  .  .  .  O  O  O  O 21 20  .
  .  .  .  .  .  .  . 22  .  .


<br><br>

<hr>

### <font color="blue">Implementation</font>

Implement breadth first search below. Here is an example of what my implementation outputs, though yours may differ:

```
  *  *  *  *  *  *  *  *  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  *  *  .  .
         is_explored	173
      legal_location	156
             explore	40
                draw	40
          found_goal	40
explored 40 cells          
  *  *  *  *  *  *  *  *  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  *  *  3  *  *  *  O  .  .  .
  *  *  4  O  O  O  O  .  .  .
  *  *  5  6  7  8  9 10  .  .
```


In [63]:
from collections import deque

def breadth_first_search(board):
    location = board.start_location
    parent = None
    visited, queue = set(), [(location, parent)]
    while not board.found_goal() and queue:
        location, parent = queue.pop(0)  # The only different is we use queue, means we pick the first element in queue
        if location not in visited:
            board.explore(location, parent)
            parent = location
            
            visited.add(location)
            queue.extend([(loc,parent) for loc in (set(board.neighbors(location))-visited)])
            
board = example_board()
breadth_first_search(board)
board.call_stats()
board.draw_best_path()

  *  *  *  *  *  *  *  *  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  *  *  .  .
      legal_location	160
          found_goal	59
             explore	40
                draw	40
explored 40 cells
  *  *  *  *  *  *  *  *  .  .
  *  *  *  O  O  O  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  *  *  3  *  *  *  O  .  .  .
  *  *  4  O  O  O  O  .  .  .
  *  *  5  6  7  8  9 10  .  .


<br><br>

<hr>

### <font color="blue">Question</font>

Is `bfs` guaranteed to find the optimal (shortest) path? Why or why not?

<hr>
<br><br>


**answer here**
We say that BFS is the algorithm to use if we want to find the shortest path in an undirected, unweighted graph. The claim for BFS is that the first time a node is discovered during the traversal, that distance from the source would give us the shortest path.With a weighted graph, the first discovery of a node during traversal does not guarantee the shortest path for that node. Breadth first search has no way of knowing if a particular discovery of a node would give us the shortest path to that node. And so, the only possible way for BFS (or DFS) to find the shortest path in a weighted graph is to search the entire graph and keep recording the minimum distance from source to the destination vertex.
<br><br><br><br>









<br><br>

<hr>

### <font color="blue">Question</font>

How do the memory requirements of dfs and bfs differ?

<hr>
<br><br>


**answer here**
BFS uses a larger amount of memory because it expands all children of a vertex and keeps them in memory. It stores the pointers to a level’s child nodes while searching each level to remember where it should go when it reaches a leaf node. DFS has much lower memory requirements than BFS because it iteratively expands only one child until it cannot proceed anymore and backtracks thereafter. It has to remember a single path with unexplored nodes.
<br><br><br><br>









Next, we'll look at greedy and A* search algorithms. To implement these, we need to prioritize the order in which we explore nodes. We'll use the priority queue implementation below to do so. While a traditional priority queue would be implemented with a heap, because we often need to update previously inserted items, we will instead use a dict. This allows us to quickly see if an item already exists, but the tradeoff is that we spend more time finding the highest priority item each time we call `get`. There are more efficient ways to do this, but this will suffice for our purposes.


In [28]:
class PriorityQueue:
    def __init__(self):
        # Dict from item to priority.
        # Smaller values mean higher priority
        self.d = {}
        
    def push(self, priority, item):
        """
        Add this item with the given priority.
        If the item already exists, update it if the new
        priority is higher than the old priority (lower value).
        """
        if item not in self.d or priority < self.d[item]:
            self.d[item] = priority
        
    def get(self):
        """
        Remove and return the (priority, item) with the highest priority (smallest value).
        """
        r = min(self.d.items(), key=lambda i: i[1])
        del self.d[r[0]]
        return (r[1], r[0])
        
    def empty(self):
        return len(self.d) == 0    
    
q = PriorityQueue()
q.push(2, 'd')
q.push(2, 'c')
q.push(3, 'b')
q.push(1, 'd') # overwrites the previous (2, 'd') call.
q.push(4, 'b') # ignored because of (3, 'b')

while not q.empty():
    print(q.get())


(1, 'd')
(2, 'c')
(3, 'b')


In [37]:
# we can add tie breakers by making the priority a tuple, where the second value 
# is used to break ties.
q = PriorityQueue()
q.push((2, 2), 'd')
q.push((2, 1), 'c')

while not q.empty():
    print(q.get())

((2, 1), 'c')
((2, 2), 'd')


<br><br>

<hr>

### <font color="blue">Question</font>

What is the worst-case, Big Oh time complexity of the `push` and `get` implementations above?

<hr>
<br><br>


**answer here**
In the Priority Queue, for the enqueue part, every time we insert a new element, we need to sort the elements. That’s O(n log n). For the dequeue part, Dequeue removes elements from the PQ. We need to find the element with the highest priority and then return that. The highest number will be first element, so that’s O(1) operation. However, we need to move the rest of the elements to fill the gap. That’s O(n).So, the Big Oh time complexity of the push and get are O(1) and O(n).
<br><br><br><br>









<br><br>

<hr>

### <font color="blue">Implementation</font>

A simple heuristic to prioritize locations is the `manhattan distance` from the location to the goal state, which is the number of moves to get to the goal state using the four directions we have. This is a type of **informed search**, since we have the additional information about where the goal is.

Implement this distance function below, given two location tuples.

In [34]:
def manhattan_distance(loc1, loc2):
    """
    Return the manhattan distance between the two given locations. Each location
    is a (x,y) tuple.
    """
    ### TODO
    return abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1])
    pass

board = example_board()
manhattan_distance(board.start_location, board.goal_location)

10

<br><br>

<hr>

### <font color="blue">Implementation</font>

Implement `greedy_search` below to prioritize exploration of the node with the smallest `manhattan_distance` to the goal. Use the `PriorityQueue` implementation above. You'll also have to keep track of the `parent` of each node (the node that let to its discovery) in order to call the `explore` method.

So, when we `push` things to the queue, the item should contain two locations, indicating the cell and its parent; i.e.:

`queue.push(cost, (location, parent))`


Note that the `greedy_search` signature accepts a `distance_fn`, which is just the name of the distance function to use. This way, we can later pass in something other than `manhattan_distance`.

Here is an example of what my implementation outputs, though yours may differ:

```
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  .  *  *  *  *  O  .  .  .
  .  .  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	76
         is_explored	56
             explore	20
                draw	20
          found_goal	20
explored 18 cells          
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  .  .  3  *  *  *  O  .  .  .
  .  .  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .
```

In [59]:
def greedy_search(board, distance_fn):
    ### TODO
    begin = board.start_location
    goal = board.goal_location
    q = PriorityQueue()
    q.push(distance_fn(begin,goal), (begin, None))

    while not q.empty() and not board.found_goal():
        pairs = q.get()
        distance = pairs[0]
        (location, parent) = pairs[1]
        if not board.is_explored(location):
            board.explore(location, parent)
            parent = location
            for nb in board.neighbors(location):
                q.push(distance_fn(nb, goal), (nb, location))  
    pass
    
board = example_board()
greedy_search(board, manhattan_distance)
board.call_stats()    
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  .  *  *  *  *  O  .  .  .
  .  .  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	72
          found_goal	28
         is_explored	27
             explore	18
                draw	18
explored 18 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  .  .  3  *  *  *  O  .  .  .
  .  .  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .


<br><br>

<hr>

### <font color="blue">Implementation</font>

Implement A* in the `astar` function below.

Sometimes the priority queue will contain ties, where $f(x) == f(y)$ To break ties in the priority queue, use the `distance_fn` passed in (h(n), which in this case is manhattan distance).

Here is an example of what my implementation outputs, though yours may differ:

```
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  .  *  *  *  *  O  .  .  .
  .  .  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	64
         is_explored	48
             explore	17
                draw	17
          found_goal	17
explored 17 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  .  .  3  *  *  *  O  .  .  .
  .  .  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .
```

In [95]:
def astar(board, distance_fn):
    ### TODO
    # g is the uniform cost
    # h is the distence
    begin = board.start_location
    goal = board.goal_location
    g = 0
    h = distance_fn(begin,goal)
    q = PriorityQueue()
    q.push((g+h,h),(begin, None))
    while not q.empty() and not board.found_goal():
        pairs = q.get()
        f, h = pairs[0]
        location, parent = pairs[1]
        g = f - h
        if not board.is_explored(location):
            board.explore(location, parent)
            parent = location
            for nb in board.neighbors(location):
                h = distance_fn(nb, goal)
                q.push((g+1+h,h),(nb,location)) # g+1 because this g is the nb, which we have spread 1 step
    pass
    
board = example_board()
astar(board, manhattan_distance)
board.call_stats()
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  .  *  *  *  *  O  .  .  .
  .  .  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	68
          found_goal	21
         is_explored	20
             explore	17
                draw	17
explored 17 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  .  .  3  *  *  *  O  .  .  .
  .  .  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .


**answer here**
We find that A* does better than greedy search,BFS, DFS
<br><br><br><br>









<br><br>

<hr>

### <font color="blue">Implementation</font>

We've said that greedy can be non-optimal. Let's find an example.

Create a board for which:
- A* finds the optimal path
- Greedy finds a suboptimal path
- A* explores more nodes than greedy.

Do so in the `hard_example_board` function below.
<hr>
<br><br>



In [80]:
def hard_example_board():
    ### TODO
    return Board(rows=7,
                 cols=10,
                 start_location=(3,0), 
                 goal_location=(6,7),
                 wall_locations=[(1,3), (1,4), (1,5), (1,6), (3,6),(4,6), (5,2), (5,3), (5,4), (5,5), (5,6)])
    pass

# run with greedy
board = hard_example_board()
greedy_search(board, manhattan_distance)
board.call_stats()
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  *  *  .  .
  *  *  *  *  *  *  O  *  .  .
  .  .  .  *  *  *  O  *  .  .
  .  .  O  O  O  O  O  *  .  .
  .  .  .  .  .  .  .  *  .  .
      legal_location	64
          found_goal	21
         is_explored	20
             explore	16
                draw	16
explored 16 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  6  7  8  .  .
  0  1  2  3  4  5  O  9  .  .
  .  .  .  *  *  *  O 10  .  .
  .  .  O  O  O  O  O 11  .  .
  .  .  .  .  .  .  . 12  .  .


In [76]:
# run with astar
board = hard_example_board()
astar(board, manhattan_distance)
board.call_stats()
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  .  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  *  *  *  *  *  O  .  .  .
  .  *  O  O  O  O  O  .  .  .
  .  *  *  *  *  *  *  *  .  .
      legal_location	76
          found_goal	24
         is_explored	23
             explore	19
                draw	19
explored 19 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  .  .  .  .  .
  0  1  *  *  *  *  O  .  .  .
  .  2  *  *  *  *  O  .  .  .
  .  3  O  O  O  O  O  .  .  .
  .  4  5  6  7  8  9 10  .  .


In [82]:
# For this example:
# - Greedy: cost is 12, explored 20 cells
# - A*: cost is 10, explored 23 cells

<br><br><br>

Below we implement an alternative distance function called `another_distance` and compare performance with using `manhattan_distance`.

In [73]:
def another_distance(loc1, loc2):
    return ((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)**.5

board = example_board()
greedy_search(board, manhattan_distance)
board.call_stats()
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  .  .  *  *  *  *  O  .  .  .
  .  .  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	72
          found_goal	28
         is_explored	27
             explore	18
                draw	18
explored 18 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  .  .  .  .  .  *  O  .  .  .
  0  1  2  *  *  *  O  .  .  .
  .  .  3  *  *  *  O  .  .  .
  .  .  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .


In [74]:
board = example_board()
astar(board, another_distance)
board.call_stats()
board.draw_best_path()

  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  *  *  *  *  .  .  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  *  *  *  O  .  .  .
  *  *  *  O  O  O  O  .  .  .
  .  .  *  *  *  *  *  *  .  .
      legal_location	100
          found_goal	45
         is_explored	44
             explore	25
                draw	25
explored 25 cells
  .  .  .  .  .  .  .  .  .  .
  .  .  .  O  O  O  O  .  .  .
  *  *  *  *  .  .  O  .  .  .
  0  *  *  *  *  *  O  .  .  .
  1  2  3  *  *  *  O  .  .  .
  *  *  4  O  O  O  O  .  .  .
  .  .  5  6  7  8  9 10  .  .


<br><br>

<hr>

### <font color="blue">Question</font>

- In terms of the number of cells explored, is the `another_distance` function more efficient, less efficient, or the same as `manhattan_distance`. Why?

- Is `another_distance` admissible? consistent? Why/Why not?

<hr>
<br><br>


**answer here**
For the manhattan_distance we explored 27 cells, while for another_distancewe explored 44 cells. So, the another_distance function is less efficient. Because the weight to each direction is the same and this is a grid. So, actually the manhattan_distance is the number of step and each step has the same cost. Then, manhattan_distance is more appropriate than another_distance.


The another_distance is admissible. Because in this case the true optimal forward cost is the manhattan_distance and the euclidean distance between two point is always smaller than the manhattan_distance.

Also, the another_distance is consistent. Assume there are two point A,C. h() is the euclidean distance. Then from the property of triangle, h(A)-h(C)<h(A-C).h(A-C) is the euclidean distance between A and C.So, h(A-C) is less than the manhattan_distance between which is also the real cost between A and C(cost(A,C)). So,  h(A)-h(C)<h(A-C)<cost(A,C). Then, it is consistent


<br><br><br><br>









<br><br>

<hr>

### <font color="blue">Question</font>

Suppose we modify our maze as follows. For some unknown number of cells, when the agent explores it, it is immediately warped to another, possibly distant, part of the board. Is the manhattan distance admissible in this setting? Why or why not? If not, provide a counter example.

<hr>
<br><br>


**answer here**
The manhattan distance is not admissible in this case. Because since the for some unknown number of cells, when the agent explores it, it is immediately warped to another, possibly distant, part of the board. So, it may make the real distance shorter than previous. Then, the manhattan distance may shorter than the real distance and it will not be admissible.
<br><br><br><br>









<br><br><br>

## <font color="blue">Game playing</font>

For the final part of this assignment, we'll implement a simple agent to play Tic Tac Toe.

Below, we've implemented `TTTBoard` to store the game state.

In [60]:
import copy

X = 'X'
O = 'O'
BLANK = '.'
DRAW = 'DRAW'

class TTTBoard:
    def __init__(self):
        self.cells = [[BLANK] * 3 for j in range(3)]
        self.to_move = X # X moves first.
        
    def draw(self):
        for i in range(3):
            print(''.join(self.cells[i]))
            
    def move(self, location):
        """
        Make a move. The player whose turn it is will be assigned to the given location.
        Note that we return a new copy of the board so that we can search for winning moves.
        """
        ttt = TTTBoard()
        ttt.cells = copy.deepcopy(self.cells)
        ttt.cells[location[0]][location[1]] = self.to_move
        ttt.to_move = X if self.to_move == O else O
        return ttt
        
    def winner(self):
        """
        Returns X or O if they have a winning position, 
        or return DRAW if there are no more moves and no one
        has one. Otherwise, return None.
        """
        for i in range(3):
            # check columns
            winner = self.match((i,0), (i,1), (i,2))
            if winner:
                return winner
            # check rows
            winner = self.match((0,i), (1,i), (2,i))
            if winner:
                return winner
        # check diagonals
        winner = self.match((0,0), (1,1), (2,2))
        if winner:
            return winner
        winner = self.match((2,0), (1,1), (0,2))
        if winner:
            return winner
        if len(self.open_squares()) == 0:
            return DRAW
        return None
    
    def match(self, a, b, c):
        # do values in these cells match? (and are not blank)
        if (self.cells[a[0]][a[1]] == self.cells[b[0]][b[1]] and
            self.cells[b[0]][b[1]] == self.cells[c[0]][c[1]] and
            self.cells[a[0]][a[1]] != BLANK):
            return self.cells[a[0]][a[1]]    
    
    def utility(self, player):
        """
        Return the utility of this board for the given player.
        1 if win, -1 if lose, 0 otherwise.
        """
        w = self.winner()
        if w == player:
            return 1
        elif w == DRAW or w is None:
            return 0
        else:
            return -1
        
    def open_squares(self):
        """
        Return a list of (x,y) tuples for the remaining
        blank squares.
        """
        result = []
        for i in range(3):
            for j in range(3):
                if self.cells[i][j] == BLANK:
                    result.append((i,j))
        return result
            

ttt = TTTBoard()
ttt.draw()

...
...
...


In [51]:
# a few example moves.
ttt = ttt.move((0,0))
ttt = ttt.move((1,1))
ttt = ttt.move((2,2))
ttt.draw()
print('winner=%s' % ttt.winner())
print('X utility=%d  O utility=%d' % (ttt.utility(X), ttt.utility(O)))

X..
.O.
..X
winner=None
X utility=0  O utility=0


In [52]:
# make X win...
ttt = ttt.move((0,1))
ttt = ttt.move((1,0))
ttt = ttt.move((0,2))
ttt = ttt.move((2,0))
ttt.draw()
print('winner=%s' % ttt.winner())
print('X utility=%d  O utility=%d' % (ttt.utility(X), ttt.utility(O)))

XOO
XO.
X.X
winner=X
X utility=1  O utility=-1


Now we need to implement some strategies to decide which square to pick next.

Below is an implementation of the main loop, `play_game`, which takes in the board, as well as a dict of strategies, one per player.

We first show how this works using a random player.

In [62]:
# let's play randomly.
def play_game(ttt, strategies):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    while ttt.winner() is  None:
        player = ttt.to_move
        move = strategies[player](ttt)
        ttt = ttt.move(move)
        clear_output(wait=True)
        ttt.draw()
        time.sleep(.5)
    print('result is: %s' % ttt.winner())

def random_strategy(ttt):
    return random.choice(ttt.open_squares())
    
random.seed(42)

ttt = TTTBoard()
play_game(ttt, {X: random_strategy, O:random_strategy})

OXX
XOO
OXX
result is: DRAW


<br><br>

<hr>

### <font color="blue">Implementation</font>

Implement `minimax_search` below, which takes in a board, runs the minimax algorithm, and returns the best move found, which is an (x,y) tuple of the location to move to.

Recall that the `TTTBoard` has a `.to_move` attribute, which indicates whose turn it is.
<hr>
<br><br>



In [99]:
def minimax(ttt):
    """Search game tree to determine best move; return (value, move) pair."""
    ### TODO
    def subminimax(ttt, MaximizingPlayer = True):
        if MaximizingPlayer == True:
            player = ttt.to_move
        else:
            player = O if ttt.to_move == X else X

        if ttt.winner() != None:
            return ttt.utility(player), None

        if MaximizingPlayer:
            v = - math.inf
            move = None
            for child in ttt.open_squares():
                u, _ = subminimax(ttt.move(child), False)
                if u > v:
                    v = u
                    move = child
            return v, move  

        else:
            v = math.inf
            move = None
            for child in ttt.open_squares():
                u, _ = subminimax(ttt.move(child), True)
                if u < v:
                    v = u
                    move = child
            return v, move
    return subminimax(ttt)[1]

ttt = TTTBoard()
chosen_move = minimax(ttt)
print(chosen_move)

(0, 0)


In [92]:
# let's make sure minimax doesn't lose to a random player.
random.seed(42)
ttt = TTTBoard()
play_game(ttt, {X: minimax, O:random_strategy})

XOO
XXX
.O.
result is: X


In [93]:
# ...and that minimax draws against itself.
ttt = TTTBoard()
play_game(ttt, {X: minimax, O:minimax})

XXO
OOX
XOX
result is: DRAW


<br><br>

<hr>

### <font color="blue">Question</font>

Notice that `TTTBoard.move` creates a entirely new copy of the `TTTBoard` object. This is to ensure we can "play out" different paths in the search tree independently.

Given that we have a 3x3 board, what is the worst-case big Oh **space** complexity of minimax?

<hr>
<br><br>


**answer here**
The space complexity is O(bm), where b is the number of legal moves at each point and m is the maximum depth of the tree. The worst case is we explore all cell of this 3x3 board, whose Oh space complexity is O(9^2)

<br><br><br><br>









<br><br>

<hr>

### <font color="blue">Implementation (graduate students only)</font>

Implement `alpha_beta` below, which performs alpha/beta pruning.
<hr>
<br><br>



In [97]:
def alpha_beta(ttt):
    ### TODO
    def sub_alphabeta(ttt,alpha,beta,player):
        c_util = ttt.utility(player)
        if c_util != 0:
            return None, c_util
        op_sqs = ttt.open_squares()
        if len(op_sqs) == 0:
            return None, 0
        
        if player == ttt.to_move:
            val, move = -math.inf, None
            for m in op_sqs:
                v = sub_alphabeta(ttt.move(m),alpha,beta,player)[1]
                if val < v:
                    val = v
                    move = m
                alpha = max(alpha,val)
                if val >= beta:
                    break
            return move,val
        else:
            val, move = math.inf, None
            for m in op_sqs:
                v = sub_alphabeta(ttt.move(m),alpha,beta,player)[1]
                if val > v:
                    val = v
                    move = m
                beta = min(beta,val)
                if val <= alpha:
                    break
            return move,val
    return sub_alphabeta(ttt,-math.inf,math.inf,ttt.to_move)

    pass
    ###

<br><br>

<hr>

### <font color="blue">Question</font>

Give an approximation of the total number of tic-tac-toe games that are possible to be played.











In [85]:
9*8*7*6*5*4*3*2

362880

**answer here**
It should be 9! games, which is 362880
<br><br><br><br>

<br><br>

<hr>

### <font color="blue">Question</font>

Currently, all non-terminal games are given a utility of 0. We may be able to speed up search if we had a heuristic that quantified the value of a certain position of the board. For example, for player X, this board:

```
O.X
..X
...
```

is probably more valuable than this board:

```
O.X
X..
...
```

Provide a (mathematical) function that takes a board and player as input and returns a score for the value of that board to the player. (No need to code this one.)









**answer here**
In one board, there are 8 line, including 3 rows, 3 columns, 2 hypotenuse.In each line, we can compute a utility for X: 
- When there is 2X and the left is a blank, u=2; 
- When there is 1X and the left are blank, u=1; 
- Otherwise, u=0
Then, we add this utility for all these 8 line and get a sum utility. When this value is large, it means X are more likely to win. So, the input is a board and player, the output can show how likely this player are going to win.
<br><br><br><br>

<br><br>

<hr>

### <font color="blue">Question</font>

Describe how you would use this function to improve the efficiency of your minimax agent. Will it still play optimally? Why or why not?








**answer here**
We can add this value to our previous utility in minimax agent. I think we should give some weight before we add this to the previous utility, otherwise, it will not be optimal. Because if we just add this value, it may happen some cases like 2X and the left is a blank is better than 3X in a line.
<br><br><br><br>